Ön hazırlık (kütüphane kurulumu)

In [1]:
# Gerekli kütüphaneleri yükleyelim (Colab hücresine)
!pip install cirq qiskit qutip numpy matplotlib --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 77.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.8/670.8 kB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 430.5/430.5 kB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 103.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.8/31.8 MB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 89.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 69.7 MB/s eta 0:00:00


1. Cirq ile 3-Qubit QFT Devresi

In [2]:
import cirq
import numpy as np

# Qubitleri tanımla
q0, q1, q2 = cirq.LineQubit.range(3)

# QFT devresi
circuit = cirq.Circuit()

# QFT adımları
# 1. qubit (q0)
circuit.append(cirq.H(q0))
circuit.append(cirq.CZ(q1, q0)**(1/2))  # R2
circuit.append(cirq.CZ(q2, q0)**(1/4))  # R3

# 2. qubit (q1)
circuit.append(cirq.H(q1))
circuit.append(cirq.CZ(q2, q1)**(1/2))  # R2

# 3. qubit (q2)
circuit.append(cirq.H(q2))

# Bit reversal (swap q0 <-> q2)
circuit.append(cirq.SWAP(q0, q2))

# Devreyi yazdır
print("Cirq QFT Circuit:")
print(circuit)

# Simülasyon
sim = cirq.Simulator()
result = sim.simulate(circuit)
print("\nFinal state vector:")
print(np.round(result.final_state_vector, 3))


Cirq QFT Circuit:
                  ┌───────┐
0: ───H───@────────@──────────────────────×───
          │        │                      │
1: ───────@^0.5────┼─────H────@───────────┼───
                   │          │           │
2: ────────────────@^0.25─────@^0.5───H───×───
                  └───────┘

Final state vector:
[0.354+0.j 0.354+0.j 0.354+0.j 0.354+0.j 0.354+0.j 0.354+0.j 0.354+0.j
 0.354+0.j]


2. Qiskit ile 3-Qubit QFT Devresi

In [3]:
from qiskit import QuantumCircuit, Aer, transpile, execute
from numpy import pi

# 3-qubitlik bir QFT devresi
qc = QuantumCircuit(3)

# QFT adımları
qc.h(0)
qc.cp(pi/2, 1, 0)  # R2
qc.cp(pi/4, 2, 0)  # R3

qc.h(1)
qc.cp(pi/2, 2, 1)  # R2

qc.h(2)

# Bit reversal (swap q0 <-> q2)
qc.swap(0, 2)

# Devreyi çizdir
print("Qiskit QFT Circuit:")
print(qc.draw())

# Simülasyon
backend = Aer.get_backend('statevector_simulator')
job = execute(transpile(qc, backend), backend)
statevector = job.result().get_statevector()
print("\nFinal state vector:")
print(statevector)


ImportError: cannot import name 'Aer' from 'qiskit' (/usr/local/lib/python3.12/dist-packages/qiskit/__init__.py)

3. QuTiP ile 3-Qubit QFT Uygulaması

In [4]:
from qutip import qeye, tensor, Qobj
import numpy as np

def qft_matrix(n):
    """n qubitlik QFT matrisi oluşturur"""
    N = 2**n
    omega = np.exp(2j * np.pi / N)
    mat = np.array([[omega**(j*k) for k in range(N)] for j in range(N)]) / np.sqrt(N)
    return Qobj(mat)

# 3-qubit QFT matrisi
QFT3 = qft_matrix(3)

# Giriş durumu |101> örneği
state = tensor([basis(2,1), basis(2,0), basis(2,1)])
output = QFT3 * state

print("QFT(3) Matrix:\n", QFT3)
print("\nInput state |101> → Output:")
print(output)


NameError: name 'basis' is not defined